<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Testing" data-toc-modified-id="Testing-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Testing</a></span><ul class="toc-item"><li><span><a href="#The-go-test-Tool" data-toc-modified-id="The-go-test-Tool-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>The go test Tool</a></span></li><li><span><a href="#Test-Functions" data-toc-modified-id="Test-Functions-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Test Functions</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Rewrite-the-functions" data-toc-modified-id="Rewrite-the-functions-11.2.0.1"><span class="toc-item-num">11.2.0.1&nbsp;&nbsp;</span>Rewrite the functions</a></span></li></ul></li><li><span><a href="#Table-driven-testing" data-toc-modified-id="Table-driven-testing-11.2.1"><span class="toc-item-num">11.2.1&nbsp;&nbsp;</span>Table-driven testing</a></span></li><li><span><a href="#Randomized-Testing" data-toc-modified-id="Randomized-Testing-11.2.2"><span class="toc-item-num">11.2.2&nbsp;&nbsp;</span>Randomized Testing</a></span></li><li><span><a href="#Testing-a-Command" data-toc-modified-id="Testing-a-Command-11.2.3"><span class="toc-item-num">11.2.3&nbsp;&nbsp;</span>Testing a Command</a></span></li><li><span><a href="#White-Box-Testing" data-toc-modified-id="White-Box-Testing-11.2.4"><span class="toc-item-num">11.2.4&nbsp;&nbsp;</span>White-Box Testing</a></span></li></ul></li><li><span><a href="#Coverage" data-toc-modified-id="Coverage-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Coverage</a></span></li><li><span><a href="#Benchmark-Functions" data-toc-modified-id="Benchmark-Functions-11.4"><span class="toc-item-num">11.4&nbsp;&nbsp;</span>Benchmark Functions</a></span></li><li><span><a href="#Profiling" data-toc-modified-id="Profiling-11.5"><span class="toc-item-num">11.5&nbsp;&nbsp;</span>Profiling</a></span></li><li><span><a href="#Example-Functions" data-toc-modified-id="Example-Functions-11.6"><span class="toc-item-num">11.6&nbsp;&nbsp;</span>Example Functions</a></span></li><li><span><a href="#Fuzz-Testing" data-toc-modified-id="Fuzz-Testing-11.7"><span class="toc-item-num">11.7&nbsp;&nbsp;</span>Fuzz Testing</a></span></li></ul></li></ul></div>

# Testing

## The go test Tool

In [ ]:
!go help test

In [3]:
!go help testflag

The 'go test' command takes both flags that apply to 'go test' itself
and flags that apply to the resulting test binary.

Several of the flags control profiling and write an execution profile
suitable for "go tool pprof"; run "go tool pprof -h" for more
information. The --alloc_space, --alloc_objects, and --show_bytes
options of pprof control how the information is presented.

The following flags are recognized by the 'go test' command and
control the execution of any test:

	-bench regexp
	    Run only those benchmarks matching a regular expression.
	    By default, no benchmarks are run.
	    To run all benchmarks, use '-bench .' or '-bench=.'.
	    The regular expression is split by unbracketed slash (/)
	    characters into a sequence of regular expressions, and each
	    part of a benchmark's identifier must match the corresponding
	    element in the sequence, if any. Possible parents of matches
	    are run with b.N=1 to identify sub-benchmarks. For example,
	    given -bench=X/

[https://pkg.go.dev/testing](https://pkg.go.dev/testing)

## Test Functions

In [4]:
import "unicode"
func IsPalindrome(s string) bool {
	var letters []rune
	for _, r := range s {
		if unicode.IsLetter(r) {
			letters = append(letters, unicode.ToLower(r))
		}
	}
	for i := range letters {
		if letters[i] != letters[len(letters)-1-i] {
			return false
		}
	}
	return true
}

In [6]:
//package word

import "testing"

func TestPalindrome(t *testing.T) {
        if !IsPalindrome("detartrated") {
                t.Error(`IsPalindrome("detartrated") = false`)
        }
        if !IsPalindrome("kayak") {
                t.Error(`IsPalindrome("kayak") = false`)
        }
}

func TestNonPalindrome(t *testing.T) {
        if IsPalindrome("palindrome") {
                t.Error(`IsPalindrome("palindrome") = true`)
        }
}

<code>
$ go test -v gopl.io/ch11/word1
</code>

In [46]:
!go test -v gopl.io/ch11/word1

=== RUN   TestPalindrome
--- PASS: TestPalindrome (0.00s)
=== RUN   TestNonPalindrome
--- PASS: TestNonPalindrome (0.00s)
=== RUN   TestFrenchPalindrome
    word_test.go:32: IsPalindrome("été") = false
--- FAIL: TestFrenchPalindrome (0.00s)
=== RUN   TestCanalPalindrome
    word_test.go:39: IsPalindrome("A man, a plan, a canal: Panama") = false
--- FAIL: TestCanalPalindrome (0.00s)
FAIL
FAIL	gopl.io/ch11/word1	0.312s
FAIL


exit status 1


In [49]:
!go test -v -run="Palin|Canal" gopl.io/ch11/word1

=== RUN   TestPalindrome
--- PASS: TestPalindrome (0.00s)
=== RUN   TestNonPalindrome
--- PASS: TestNonPalindrome (0.00s)
=== RUN   TestFrenchPalindrome
    word_test.go:32: IsPalindrome("été") = false
--- FAIL: TestFrenchPalindrome (0.00s)
=== RUN   TestCanalPalindrome
    word_test.go:39: IsPalindrome("A man, a plan, a canal: Panama") = false
--- FAIL: TestCanalPalindrome (0.00s)
FAIL
FAIL	gopl.io/ch11/word1	0.087s
FAIL


exit status 1


#### Rewrite the functions

In [39]:
// IsPalindrome reports whether s reads the same forward and backward.
// Letter case is ignored, as are non-letters.
func IsPalindrome(s string) bool {
        //var letters []rune
        letters := make([]rune,0, len(s))
        for _, r := range s {
                if unicode.IsLetter(r) {
                        letters = append(letters, unicode.ToLower(r))
                }
        }
        //for i := range letters {
        n := len(letters)/2
        for i :=0; i<n; i++ {
                if letters[i] != letters[len(letters)-1-i] {
                        return false
                }
        }
        return true
}


### Table-driven testing

In [41]:
import (
        "fmt"
        "math/rand"
        "time"
)

//!+bench

import "testing"

//!-bench

//!+test
func TestIsPalindrome(t *testing.T) {
        var tests = []struct {
                input string
                want  bool
        }{
                {"", true},
                {"a", true},
                {"aa", true},
                {"ab", false},
                {"kayak", true},
                   {"detartrated", true},
                {"A man, a plan, a canal: Panama", true},
                {"Evil I did dwell; lewd did I live.", true},
                {"Able was I ere I saw Elba", true},
                {"été", true},
                {"Et se resservir, ivresse reste.", true},
                {"palindrome", false}, // non-palindrome
                {"desserts", false},   // semi-palindrome
        }
        for _, test := range tests {
                if got := IsPalindrome(test.input); got != test.want {
                        t.Errorf("IsPalindrome(%q) = %v", test.input, got)
                }
        }
}


In [50]:
!go test -v gopl.io/ch11/word2

=== RUN   TestIsPalindrome
--- PASS: TestIsPalindrome (0.00s)
=== RUN   TestRandomPalindromes
    word_test.go:92: Random seed: 1702454498978667000
--- PASS: TestRandomPalindromes (0.00s)
=== RUN   ExampleIsPalindrome
--- PASS: ExampleIsPalindrome (0.00s)
PASS
ok  	gopl.io/ch11/word2	(cached)


### Randomized Testing

In [42]:
import "math/rand"
func randomPalindrome(rng *rand.Rand) string {
	n := rng.Intn(25) // random length up to 24
	runes := make([]rune, n)
	for i := 0; i < (n+1)/2; i++ {
		r := rune(rng.Intn(0x1000)) // random rune up to '\u0999'
		runes[i] = r
		runes[n-1-i] = r
	}
	return string(runes)
}

In [43]:
import "testing"
import "time"
func TestRandomPalindromes(t *testing.T) {
	// Initialize a pseudo-random number generator.
	seed := time.Now().UTC().UnixNano()
	t.Logf("Random seed: %d", seed)
	rng := rand.New(rand.NewSource(seed))

	for i := 0; i < 1000; i++ {
		p := randomPalindrome(rng)
		if !IsPalindrome(p) {
			t.Errorf("IsPalindrome(%q) = false", p)
		}
	}
}

In [16]:
!go test -v -run="Random" gopl.io/ch11/word2

=== RUN   TestRandomPalindromes
    word_test.go:92: Random seed: 1702454601585470000
--- PASS: TestRandomPalindromes (0.00s)
PASS
ok  	gopl.io/ch11/word2	0.099s


### Testing a Command

In [44]:
//gopl.io/ch11/echo
import (
	"flag"
	"fmt"
	"io"
	"os"
	"strings"
)

var (
	n = flag.Bool("n", false, "omit trailing newline")
	s = flag.String("s", " ", "separator")
)
var out io.Writer = os.Stdout // modified during testing

func main() {
	flag.Parse()
	if err := echo(!*n, *s, flag.Args()); err != nil {
		fmt.Fprintf(os.Stderr, "echo: %v\n", err)
		os.Exit(1)
	}
}

func echo(newline bool, sep string, args []string) error {
	fmt.Fprint(out, strings.Join(args, sep))
	if newline {
		fmt.Fprintln(out)
	}
	return nil
}

In [45]:
// Test of echo command.  Run with: go test gopl.io/ch11/echo

//!+
//package main

import (
	"bytes"
	"fmt"
	"testing"
)

func TestEcho(t *testing.T) {
	var tests = []struct {
		newline bool
		sep     string
		args    []string
		want    string
	}{
		{true, "", []string{}, "\n"},
		{false, "", []string{}, ""},
		{true, "\t", []string{"one", "two", "three"}, "one\ttwo\tthree\n"},
		{true, ",", []string{"a", "b", "c"}, "a,b,c\n"},
		{false, ":", []string{"1", "2", "3"}, "1:2:3"},
	}

	for _, test := range tests {
		descr := fmt.Sprintf("echo(%v, %q, %q)",
			test.newline, test.sep, test.args)

		out = new(bytes.Buffer) // captured output
		if err := echo(test.newline, test.sep, test.args); err != nil {
			t.Errorf("%s failed: %v", descr, err)
			continue
		}
		got := out.(*bytes.Buffer).String()
		if got != test.want {
			t.Errorf("%s = %q, want %q", descr, got, test.want)
		}
	}
}


In [51]:
!go test -v gopl.io/ch11/echo

=== RUN   TestEcho
--- PASS: TestEcho (0.00s)
PASS
ok  	gopl.io/ch11/echo	(cached)


### White-Box Testing

In [19]:
//gopl.io/ch11/storage1
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 311.

// Package storage is part of a hypothetical cloud storage server.
//!+main
//package storage

import (
	"fmt"
	"log"
	"net/smtp"
)

var usage = make(map[string]int64)

func bytesInUse(username string) int64 { return usage[username] }

// Email sender configuration.
// NOTE: never put passwords in source code!
const sender = "notifications@example.com"
const password = "correcthorsebatterystaple"
const hostname = "smtp.example.com"

const template = `Warning: you are using %d bytes of storage, %d%% of your quota.`

func CheckQuota(username string) {
	used := bytesInUse(username)
	const quota = 1000000000 // 1GB
	percent := 100 * used / quota
	if percent < 90 {
		return // OK
	}
	msg := fmt.Sprintf(template, used, percent)
	auth := smtp.PlainAuth("", sender, password, hostname)
	err := smtp.SendMail(hostname+":587", auth, sender,
		[]string{username}, []byte(msg))
	if err != nil {
		log.Printf("smtp.SendMail(%s) failed: %s", username, err)
	}
}

//!-main

In [21]:
//gopl.io/ch11/storage2
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 312.

// Package storage is part of a hypothetical cloud storage server.
//package storage

import (
	"fmt"
	"log"
	"net/smtp"
)

var usage = make(map[string]int64)

func bytesInUse(username string) int64 { return usage[username] }

// E-mail sender configuration.
// NOTE: never put passwords in source code!
const sender = "notifications@example.com"
const password = "correcthorsebatterystaple"
const hostname = "smtp.example.com"

const template = `Warning: you are using %d bytes of storage, %d%% of your quota.`

//!+factored
var notifyUser = func(username, msg string) {
	auth := smtp.PlainAuth("", sender, password, hostname)
	err := smtp.SendMail(hostname+":587", auth, sender,
		[]string{username}, []byte(msg))
	if err != nil {
		log.Printf("smtp.SendMail(%s) failed: %s", username, err)
	}
}

func CheckQuota(username string) {
	used := bytesInUse(username)
	const quota = 1000000000 // 1GB
	percent := 100 * used / quota
	if percent < 90 {
		return // OK
	}
	msg := fmt.Sprintf(template, used, percent)
	notifyUser(username, msg)
}

//!-factored

In [22]:
//gopl.io/ch11/quota_test.go
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

//!+test
//package storage

import (
	"strings"
	"testing"
)

func TestCheckQuotaNotifiesUser(t *testing.T) {
	var notifiedUser, notifiedMsg string
	notifyUser = func(user, msg string) {
		notifiedUser, notifiedMsg = user, msg
	}

	const user = "joe@example.org"
	usage[user] = 980000000 // simulate a 980MB-used condition

	CheckQuota(user)
	if notifiedUser == "" && notifiedMsg == "" {
		t.Fatalf("notifyUser not called")
	}
	if notifiedUser != user {
		t.Errorf("wrong user (%s) notified, want %s",
			notifiedUser, user)
	}
	const wantSubstring = "98% of your quota"
	if !strings.Contains(notifiedMsg, wantSubstring) {
		t.Errorf("unexpected notification message <<%s>>, "+
			"want substring %q", notifiedMsg, wantSubstring)
	}
}

//!-test

/*
//!+defer
func TestCheckQuotaNotifiesUser(t *testing.T) {
	// Save and restore original notifyUser.
	saved := notifyUser
	defer func() { notifyUser = saved }()
	// Install the test's fake notifyUser.
	var notifiedUser, notifiedMsg string
	notifyUser = func(user, msg string) {
		notifiedUser, notifiedMsg = user, msg
	}
	// ...rest of test...
}
//!-defer
*/

In [24]:
!go test -v gopl.io/ch11/storage2

=== RUN   TestCheckQuotaNotifiesUser
--- PASS: TestCheckQuotaNotifiesUser (0.00s)
PASS
ok  	gopl.io/ch11/storage2	(cached)


## Coverage

In [29]:
import ("fmt";"math";"testing")
func TestEval(t *testing.T) {
        tests := []struct {
                expr string
                env  Env
                want string
        }{
                {"sqrt(A / pi)", Env{"A": 87616, "pi": math.Pi}, "167"},
                {"pow(x, 3) + pow(y, 3)", Env{"x": 12, "y": 1}, "1729"},
                {"pow(x, 3) + pow(y, 3)", Env{"x": 9, "y": 10}, "1729"},
                {"5 / 9 * (F - 32)", Env{"F": -40}, "-40"},
                {"5 / 9 * (F - 32)", Env{"F": 32}, "0"},
                {"5 / 9 * (F - 32)", Env{"F": 212}, "100"},
                //!-Eval
                // additional tests that don't appear in the book
                {"-1 + -x", Env{"x": 1}, "-2"},
                {"-1 - x", Env{"x": 1}, "-2"},
                //!+Eval
        }
        var prevExpr string
        for _, test := range tests {
                // Print expr only when it changes.
                if test.expr != prevExpr {
                        fmt.Printf("\n%s\n", test.expr)
                        prevExpr = test.expr
                }
                expr, err := Parse(test.expr)
                if err != nil {
                        t.Error(err) // parse error
                        continue
                }
                got := fmt.Sprintf("%.6g", expr.Eval(test.env))
                fmt.Printf("\t%v => %s\n", test.env, got)
                if got != test.want {
                        t.Errorf("%s.Eval() in %v = %q, want %q\n",
                                test.expr, test.env, got, test.want)
                }
        }
}

# gonb_ae710ac8 

 

 Cell[29]: Line 5 
 ./main.go:87:22: undefined: Env
 
func TestEval(t *testing.T) {
 tests := []struct {
 expr string
 env Env
 want string
 }{

 

 

 Cell[29]: Line 8 
 ./main.go:90:34: undefined: Env
 
 env Env
 want string
 }{
 {"sqrt(A / pi)", Env{"A": 87616, "pi": math.Pi}, "167"},
 {"pow(x, 3) + pow(y, 3)", Env{"x": 12, "y": 1}, "1729"},
 {"pow(x, 3) + pow(y, 3)", Env{"x": 9, "y": 10}, "1729"},

 

 

 Cell[29]: Line 9 
 ./main.go:91:43: undefined: Env
 
 want string
 }{
 {"sqrt(A / pi)", Env{"A": 87616, "pi": math.Pi}, "167"},
 {"pow(x, 3) + pow(y, 3)", Env{"x": 12, "y": 1}, "1729"},
 {"pow(x, 3) + pow(y, 3)", Env{"x": 9, "y": 10}, "1729"},
 {"5 / 9 * (F - 32)", Env{"F": -40}, "-40"},

 

 

 Cell[29]: Line 10 
 ./main.go:92:43: undefined: Env
 
 }{
 {"sqrt(A / pi)", Env{"A": 87616, "pi": math.Pi}, "167"},
 {"pow(x, 3) + pow(y, 3)", Env{"x": 12, "y": 1}, "1729"},
 {"pow(x, 3) + pow(y, 3)", Env{"x": 9, "y": 10}, "1729"},
 {"5 / 9 * (F - 32)", Env{"F": -40}, "-40"},
 {"5 / 9 * (F - 32)", Env{"F": 32}, "0"},

 

 

 Cell[29]: Line 11 
 ./main.go:93:38: undefined: Env
 
 {"sqrt(A / pi)", Env{"A": 87616, "pi": math.Pi}, "167"},
 {"pow(x, 3) + pow(y, 3)", Env{"x": 12, "y": 1}, "1729"},
 {"pow(x, 3) + pow(y, 3)", Env{"x": 9, "y": 10}, "1729"},
 {"5 / 9 * (F - 32)", Env{"F": -40}, "-40"},
 {"5 / 9 * (F - 32)", Env{"F": 32}, "0"},
 {"5 / 9 * (F - 32)", Env{"F": 212}, "100"},

 

 

 Cell[29]: Line 12 
 ./main.go:94:38: undefined: Env
 
 {"pow(x, 3) + pow(y, 3)", Env{"x": 12, "y": 1}, "1729"},
 {"pow(x, 3) + pow(y, 3)", Env{"x": 9, "y": 10}, "1729"},
 {"5 / 9 * (F - 32)", Env{"F": -40}, "-40"},
 {"5 / 9 * (F - 32)", Env{"F": 32}, "0"},
 {"5 / 9 * (F - 32)", Env{"F": 212}, "100"},
 //!-Eval

 

 

 Cell[29]: Line 13 
 ./main.go:95:38: undefined: Env
 
 {"pow(x, 3) + pow(y, 3)", Env{"x": 9, "y": 10}, "1729"},
 {"5 / 9 * (F - 32)", Env{"F": -40}, "-40"},
 {"5 / 9 * (F - 32)", Env{"F": 32}, "0"},
 {"5 / 9 * (F - 32)", Env{"F": 212}, "100"},
 //!-Eval
 // additional tests that don't appear in the book

 

 

 Cell[29]: Line 16 
 ./main.go:98:29: undefined: Env
 
 {"5 / 9 * (F - 32)", Env{"F": 212}, "100"},
 //!-Eval
 // additional tests that don't appear in the book
 {"-1 + -x", Env{"x": 1}, "-2"},
 {"-1 - x", Env{"x": 1}, "-2"},
 //!+Eval

 

 

 Cell[29]: Line 17 
 ./main.go:99:28: undefined: Env
 
 //!-Eval
 // additional tests that don't appear in the book
 {"-1 + -x", Env{"x": 1}, "-2"},
 {"-1 - x", Env{"x": 1}, "-2"},
 //!+Eval
 }

 

 

 Cell[29]: Line 27 
 ./main.go:109:30: undefined: Parse
 
 fmt.Printf("\n%s\n", test.expr)
 prevExpr = test.expr
 }
 expr, err := Parse(test.expr)
 if err != nil {
 t.Error(err) // parse error

 

 

 Cell[29]: Line 27 
 ./main.go:109:30: too many errors
 
 fmt.Printf("\n%s\n", test.expr)
 prevExpr = test.expr
 }
 expr, err := Parse(test.expr)
 if err != nil {
 t.Error(err) // parse error

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_ae710ac8/gonb_ae710ac8": exit status 1

<code>
$ go test -v -run=Coverage gopl.io/ch7/eval 
=== RUN   TestCoverage
--- PASS: TestCoverage (0.00s)
PASS
ok      gopl.io/ch7/eval    0.011s
</code>

In [52]:
!go test -v -run=Coverage gopl.io/ch7/eval 

=== RUN   TestCoverage
--- PASS: TestCoverage (0.00s)
PASS
ok  	gopl.io/ch7/eval	(cached)


<code>
$ go test -v -run=Coverage -coverprofile=c.out gopl.io/ch7/eval
=== RUN   TestCoverage
--- PASS: TestCoverage (0.00s)
PASS
coverage: 63.8% of statements
ok      gopl.io/ch7/eval        0.018s  coverage: 63.8% of statements
</code>

In [53]:
!go test -v -run=Coverage -coverprofile=c.out gopl.io/ch7/eval

=== RUN   TestCoverage
--- PASS: TestCoverage (0.00s)
PASS
coverage: 63.8% of statements
ok  	gopl.io/ch7/eval	0.398s	coverage: 63.8% of statements


<code>
$ go tool cover -html=c.out
HTML output written to /tmp/cover854632247/coverage.html
</code>

In [54]:
!go tool cover -html=c.out

[coverage.html](cover3574731311/coverage.html)

## Benchmark Functions

In [49]:
func BenchmarkIsPalindrome(b *testing.B) {
        for i := 0; i < b.N; i++ {
                IsPalindrome("A man, a plan, a canal: Panama")
        }
}

In [50]:
//word.go.1
func IsPalindrome(s string) bool {
        var letters []rune
        for _, r := range s {
                if unicode.IsLetter(r) {
                        letters = append(letters, unicode.ToLower(r))
                }
        }
        for i := range letters {
                if letters[i] != letters[len(letters)-1-i] {
                        return false
                }
        }
        return true
}

<code>
$ rm word.go; ln -s word.go.1 word.go; go test -bench=.
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          3274288               376.9 ns/op
PASS
ok      gopl.io/ch11/word2      1.627s
</code>

In [55]:
!(pwd;cd word2;rm word.go; ln -s word.go.1 word.go; go test -bench=.)

/Users/skhuang/course/go2023/go2023/ch11
goos: darwin
goarch: arm64
pkg: ch7/ch11/word2
BenchmarkIsPalindrome-10    	 6457305	       166.3 ns/op
PASS
ok  	ch7/ch11/word2	1.671s


<code>
rm word.go; ln -s word.go.1 word.go; go test -bench=. -benchmem
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          2169936               545.5 ns/op           248 B/op          5 allocs/op
PASS
ok      gopl.io/ch11/word2      1.773s
</code>

In [56]:
!(cd word2;rm word.go; ln -s word.go.1 word.go; go test -bench=. -benchmem)

goos: darwin
goarch: arm64
pkg: ch7/ch11/word2
BenchmarkIsPalindrome-10    	 7143759	       167.0 ns/op	     248 B/op	       5 allocs/op
PASS
ok  	ch7/ch11/word2	1.453s


-----------

In [51]:
//word.go.2
func IsPalindrome(s string) bool {
        var letters []rune
        for _, r := range s {
                if unicode.IsLetter(r) {
                        letters = append(letters, unicode.ToLower(r))
                }
        }
        //for i := range letters {
        n := len(letters)/2
        for i :=0; i<n; i++ {
                if letters[i] != letters[len(letters)-1-i] {
                        return false
                }
        }
        return true
}

<code>
$ rm word.go; ln -s word.go.2 word.go; go test -bench=.
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          3230330               363.3 ns/op
PASS
ok      gopl.io/ch11/word2      1.571s
</code>

In [57]:
!(cd word2;rm word.go; ln -s word.go.2 word.go; go test -bench=.)

goos: darwin
goarch: arm64
pkg: ch7/ch11/word2
BenchmarkIsPalindrome-10    	 6391077	       164.1 ns/op
PASS
ok  	ch7/ch11/word2	1.539s


<code>
rm word.go; ln -s word.go.2 word.go; go test -bench=. -benchmem
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          3204992               364.9 ns/op           248 B/op          5 allocs/op
PASS
ok      gopl.io/ch11/word2      1.574s
    </code>

In [58]:
!(cd word2;rm word.go; ln -s word.go.2 word.go; go test -bench=. -benchmem)

goos: darwin
goarch: arm64
pkg: ch7/ch11/word2
BenchmarkIsPalindrome-10    	 7364905	       162.2 ns/op	     248 B/op	       5 allocs/op
PASS
ok  	ch7/ch11/word2	1.435s


----

In [52]:
//word.go.3
func IsPalindrome(s string) bool {
        //var letters []rune
        letters := make([]rune,0, len(s))
        for _, r := range s {
                if unicode.IsLetter(r) {
                        letters = append(letters, unicode.ToLower(r))
                }
        }
        for i := range letters {
                if letters[i] != letters[len(letters)-1-i] {
                        return false
                }
        }
        return true
}

<code>
$ rm word.go; ln -s word.go.3 word.go; go test -bench=.
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          5476538               209.7 ns/op
PASS
ok      gopl.io/ch11/word2      1.390s
</code>

In [60]:
!(cd word2;rm word.go; ln -s word.go.3 word.go; go test -bench=.)

goos: darwin
goarch: arm64
pkg: ch7/ch11/word2
BenchmarkIsPalindrome-10    	12704876	        89.96 ns/op
PASS
ok  	ch7/ch11/word2	1.330s


<code>
rm word.go; ln -s word.go.3 word.go; go test -bench=. -benchmem
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          5408686               211.7 ns/op           128 B/op          1 allocs/op
PASS
ok      gopl.io/ch11/word2      1.390s
</code>

In [61]:
!(cd word2;rm word.go; ln -s word.go.3 word.go; go test -bench=. -benchmem)

goos: darwin
goarch: arm64
pkg: ch7/ch11/word2
BenchmarkIsPalindrome-10    	12906168	        88.60 ns/op	     128 B/op	       1 allocs/op
PASS
ok  	ch7/ch11/word2	1.324s


------

In [53]:
//word.go.4
func IsPalindrome(s string) bool {
        //var letters []rune
        letters := make([]rune,0, len(s))
        for _, r := range s {
                if unicode.IsLetter(r) {
                        letters = append(letters, unicode.ToLower(r))
                }
        }
        //for i := range letters {
        n := len(letters)/2
        for i :=0; i<n; i++ {
                if letters[i] != letters[len(letters)-1-i] {
                        return false
                }
        }
        return true
}

<code>
rm word.go; ln -s word.go.4 word.go; go test -bench=.
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          5644834               191.5 ns/op
PASS
ok      gopl.io/ch11/word2      1.319s
</code>

In [62]:
!(cd word2;rm word.go; ln -s word.go.4 word.go; go test -bench=.)

goos: darwin
goarch: arm64
pkg: ch7/ch11/word2
BenchmarkIsPalindrome-10    	13778595	        85.82 ns/op
PASS
ok  	ch7/ch11/word2	2.778s


<code>
rm word.go; ln -s word.go.4 word.go; go test -bench=. -benchmem
goos: linux
goarch: amd64
pkg: gopl.io/ch11/word2
cpu: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz
BenchmarkIsPalindrome-4          5759314               194.2 ns/op           128 B/op          1 allocs/op
PASS
ok      gopl.io/ch11/word2      1.354s
</code>

In [63]:
!(cd word2;rm word.go; ln -s word.go.4 word.go; go test -bench=. -benchmem)

goos: darwin
goarch: arm64
pkg: ch7/ch11/word2
BenchmarkIsPalindrome-10    	13060276	        87.02 ns/op	     128 B/op	       1 allocs/op
PASS
ok  	ch7/ch11/word2	1.318s


## Profiling

<code>
$ ulimit -n 4000
$ go test -run=NONE -bench=ClientServerParallelTLS64 -cpuprofile=cpu.log net/http
PASS
goos: linux
goarch: amd64
pkg: net/http
cpu: Intel(R) Xeon(R) CPU           X5675  @ 3.07GHz
BenchmarkClientServerParallelTLS64-4    2021/12/21 16:37:16 http: TLS handshake error fr
om 127.0.0.1:42421: read tcp 127.0.0.1:36819->127.0.0.1:42421: use of closed network con
nection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42450: read tcp 127.0.0.1:3
6819->127.0.0.1:42450: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42463: read tcp 127.0.0.1:3
6819->127.0.0.1:42463: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42454: write tcp 127.0.0.1:
36819->127.0.0.1:42454: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42481: read tcp 127.0.0.1:3
6819->127.0.0.1:42481: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42483: read tcp 127.0.0.1:3
6819->127.0.0.1:42483: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42456: write tcp 127.0.0.1:
36819->127.0.0.1:42456: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42479: write tcp 127.0.0.1:
36819->127.0.0.1:42479: use of closed network connection
2021/12/21 16:37:16 http: TLS handshake error from 127.0.0.1:42471: write tcp 127.0.0.1:
36819->127.0.0.1:42471: use of closed network connection
</code>

In [64]:
!(ulimit -n 4000;go test -run=NONE -bench=ClientServerParallelTLS64 -cpuprofile=cpu.log net/http)

goos: darwin
goarch: arm64
pkg: net/http
BenchmarkClientServerParallelTLS64-10    	2023/12/14 16:05:15 http: TLS handshake error from 127.0.0.1:63322: read tcp 127.0.0.1:63234->127.0.0.1:63322: use of closed network connection
2023/12/14 16:05:15 http: TLS handshake error from 127.0.0.1:63243: read tcp 127.0.0.1:63234->127.0.0.1:63243: use of closed network connection
2023/12/14 16:05:15 http: TLS handshake error from 127.0.0.1:63308: write tcp 127.0.0.1:63234->127.0.0.1:63308: use of closed network connection
2023/12/14 16:05:15 http: TLS handshake error from 127.0.0.1:63297: read tcp 127.0.0.1:63234->127.0.0.1:63297: use of closed network connection
2023/12/14 16:05:15 http: TLS handshake error from 127.0.0.1:63317: read tcp 127.0.0.1:63234->127.0.0.1:63317: use of closed network connection
2023/12/14 16:05:15 http: TLS handshake error from 127.0.0.1:63246: read tcp 127.0.0.1:63234->127.0.0.1:63246: use of closed network connection
2023/12/14 16:05:15 http: TLS handshake error from 1

PASS
ok  	net/http	1.696s


<code>
go tool pprof -text -nodecount=10 ./http.test cpu.log
File: http.test
Type: cpu
Time: Dec 21, 2021 at 4:37pm (CST)
Duration: 3.95s, Total samples = 9300ms (235.73%)
Showing nodes accounting for 6210ms, 66.77% of 9300ms total
Dropped 355 nodes (cum <= 46.50ms)
Showing top 10 nodes out of 229
      flat  flat%   sum%        cum   cum%
    1840ms 19.78% 19.78%     1840ms 19.78%  math/big.addMulVVW
    1340ms 14.41% 34.19%     1340ms 14.41%  vendor/golang.org/x/crypto/curve25519.ladderstep
    1060ms 11.40% 45.59%     1080ms 11.61%  syscall.Syscall
     560ms  6.02% 51.61%     2400ms 25.81%  math/big.nat.montgomery
     550ms  5.91% 57.53%      550ms  5.91%  crypto/sha256.block
     220ms  2.37% 59.89%      440ms  4.73%  runtime.pcvalue
     180ms  1.94% 61.83%      210ms  2.26%  runtime.step
     160ms  1.72% 63.55%      160ms  1.72%  vendor/golang.org/x/crypto/curve25519.square
     150ms  1.61% 65.16%      170ms  1.83%  runtime.findObject
     150ms  1.61% 66.77%      850ms  9.14%  runtime.mallocgc
</code>

In [65]:
!go tool pprof -text -nodecount=10 ./http.test cpu.log

File: http.test
Type: cpu
Time: Dec 14, 2023 at 4:05pm (CST)
Duration: 1.56s, Total samples = 8510ms (545.51%)
Showing nodes accounting for 7620ms, 89.54% of 8510ms total
Dropped 175 nodes (cum <= 42.55ms)
Showing top 10 nodes out of 213
      flat  flat%   sum%        cum   cum%
    3430ms 40.31% 40.31%     3440ms 40.42%  syscall.syscall
     790ms  9.28% 49.59%      790ms  9.28%  runtime.madvise
     700ms  8.23% 57.81%      700ms  8.23%  runtime.kevent
     550ms  6.46% 64.28%      550ms  6.46%  runtime.usleep
     470ms  5.52% 69.80%      470ms  5.52%  math/big.addMulVVW
     410ms  4.82% 74.62%      410ms  4.82%  runtime.pthread_cond_wait
     410ms  4.82% 79.44%      410ms  4.82%  syscall.syscall6
     370ms  4.35% 83.78%      370ms  4.35%  syscall.rawSyscall
     300ms  3.53% 87.31%      780ms  9.17%  math/big.nat.montgomery
     190ms  2.23% 89.54%      190ms  2.23%  runtime.pthread_kill


## Example Functions

In [1]:
func ExampleIsPalindrome() {
        fmt.Println(IsPalindrome("A man, a plan, a canal: Panama"))
        fmt.Println(IsPalindrome("palindrome"))
        // Output:
        // true
        // false
}


# gonb_e2c12478 

 

 Cell[1]: Line 2 
 ./main.go:9:21: undefined: IsPalindrome
 
)

func ExampleIsPalindrome() {
 fmt.Println(IsPalindrome("A man, a plan, a canal: Panama"))
 fmt.Println(IsPalindrome("palindrome"))
 // Output:

 

 

 Cell[1]: Line 3 
 ./main.go:10:21: undefined: IsPalindrome
 

func ExampleIsPalindrome() {
 fmt.Println(IsPalindrome("A man, a plan, a canal: Panama"))
 fmt.Println(IsPalindrome("palindrome"))
 // Output:
 // true

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_e2c12478/gonb_e2c12478": exit status 1

## Fuzz Testing

* An installation of Go 1.18 or later. 
* A tool to edit your code. Any text editor you have will work fine.
* A command terminal. Go works well using any terminal on Linux and Mac, and on PowerShell or cmd in Windows.
* An environment that supports fuzzing. 
  * Go fuzzing with coverage instrumentation is only available on AMD64 and ARM64 architectures currently.

In [8]:
func Reverse(s string) string {
    b := []byte(s)
    for i, j := 0, len(b)-1; i < len(b)/2; i, j = i+1, j-1 {
        b[i], b[j] = b[j], b[i]
    }
    return string(b)
}

In [9]:
func main() {
    input := "The quick brown fox jumped over the lazy dog"
    rev := Reverse(input)
    doubleRev := Reverse(rev)
    fmt.Printf("original: %q\n", input)
    fmt.Printf("reversed: %q\n", rev)
    fmt.Printf("reversed again: %q\n", doubleRev)
}

original: "The quick brown fox jumped over the lazy dog"
reversed: "god yzal eht revo depmuj xof nworb kciuq ehT"
reversed again: "The quick brown fox jumped over the lazy dog"


In [10]:
!pwd
!(cd fuzz;go run .)

/Users/skhuang/course/go2023/go2023/ch11
original: "The quick brown fox jumped over the lazy dog"
reversed: "god yzal eht revo depmuj xof nworb kciuq ehT", err: <nil>
reversed again: "The quick brown fox jumped over the lazy dog", err: <nil>


In [11]:
//reverse_test.go
import (
    "testing"
)

func TestReverse(t *testing.T) {
    testcases := []struct {
        in, want string
    }{
        {"Hello, world", "dlrow ,olleH"},
        {" ", " "},
        {"!12345", "54321!"},
    }
    for _, tc := range testcases {
        rev := Reverse(tc.in)
        if rev != tc.want {
                t.Errorf("Reverse: %q, want %q", rev, tc.want)
        }
    }
}

In [12]:
!(cd fuzz; go test -run="TestReverse" -v)

=== RUN   TestReverse
--- PASS: TestReverse (0.00s)
PASS
ok  	example/fuzz	0.253s


In [13]:
//Add a fuzz test
func FuzzReverse(f *testing.F) {
    testcases := []string{"Hello, world", " ", "!12345"}
    for _, tc := range testcases {
        f.Add(tc)  // Use f.Add to provide a seed corpus
    }
    f.Fuzz(func(t *testing.T, orig string) {
        rev := Reverse(orig)
        doubleRev := Reverse(rev)
        if orig != doubleRev {
            t.Errorf("Before: %q, after: %q", orig, doubleRev)
        }
        if utf8.ValidString(orig) && !utf8.ValidString(rev) {
            t.Errorf("Reverse produced invalid UTF-8 string %q", rev)
        }
    })
}

In [14]:
!(cd fuzz; ./reset1; go test -fuzz="Fuzz" -v)

=== RUN   TestReverse
--- PASS: TestReverse (0.00s)
=== FUZZ  FuzzReverse
fuzz: elapsed: 0s, gathering baseline coverage: 0/49 completed
failure while testing seed corpus entry: FuzzReverse/d5b6b60965e92e557798e88a10d04e5fb1070907a7dc6118b367e4fb501354bd
fuzz: elapsed: 0s, gathering baseline coverage: 6/49 completed
--- FAIL: FuzzReverse (0.04s)
    --- FAIL: FuzzReverse (0.00s)
        reverse_test.go:36: Reverse produced invalid UTF-8 string "\xb1\xd4"
    
FAIL
exit status 1
FAIL	example/fuzz	0.297s


exit status 1


In [15]:
!cat fuzz/testdata/fuzz/FuzzReverse/d5b6b60965e92e557798e88a10d04e5fb1070907a7dc6118b367e4fb501354bd

go test fuzz v1
string("Ա")


In [16]:
!(cd fuzz; go test)

--- FAIL: FuzzReverse (0.00s)
    --- FAIL: FuzzReverse/d5b6b60965e92e557798e88a10d04e5fb1070907a7dc6118b367e4fb501354bd (0.00s)
        reverse_test.go:36: Reverse produced invalid UTF-8 string "\xb1\xd4"
FAIL
exit status 1
FAIL	example/fuzz	0.379s


exit status 1


<code>
f.Fuzz(func(t *testing.T, orig string) {
    rev := Reverse(orig)
    doubleRev := Reverse(rev)
    t.Logf("Number of runes: orig=%d, rev=%d, doubleRev=%d", utf8.RuneCountInString(orig), utf8.RuneCountInString(rev), utf8.RuneCountInString(doubleRev))
    if orig != doubleRev {
        t.Errorf("Before: %q, after: %q", orig, doubleRev)
    }
    if utf8.ValidString(orig) && !utf8.ValidString(rev) {
        t.Errorf("Reverse produced invalid UTF-8 string %q", rev)
    }
})
</code>

In [19]:
!(cd fuzz;./reset2; go test)

input: "Hello, world"
input: " "
input: "!12345"
input: "Hello, world"
input: "dlrow ,olleH"
input: " "
input: " "
input: "!12345"
input: "54321!"
input: "\x80"
input: "�"
input: "Ա"
input: "Ա"
--- FAIL: FuzzReverse (0.00s)
    --- FAIL: FuzzReverse/98fce631eb9c5dd541458bf71a7241469a103001c64053f3a71625b41ccaea5f (0.00s)
        reverse_test.go:32: Number of runes: orig=1, rev=1, doubleRev=1
        reverse_test.go:34: Before: "\x80", after: "�"
FAIL
exit status 1
FAIL	example/fuzz	0.172s


exit status 1


In [20]:
!(cd fuzz;./reset2; go test -run=FuzzReverse/d5b6b60965e92e557798e88a10d04e5fb1070907a7dc6118b367e4fb501354bd)

input: "Ա"
input: "Ա"
PASS
ok  	example/fuzz	0.101s


In [45]:
!(cd fuzz; ./reset2; go test -fuzz="Fuzz")

input: "Hello, world"
input: " "
input: "!12345"
fuzz: elapsed: 0s, gathering baseline coverage: 0/48 completed
failure while testing seed corpus entry: FuzzReverse/98fce631eb9c5dd541458bf71a7241469a103001c64053f3a71625b41ccaea5f
fuzz: elapsed: 0s, gathering baseline coverage: 2/48 completed
--- FAIL: FuzzReverse (0.04s)
    --- FAIL: FuzzReverse (0.00s)
        reverse_test.go:32: Number of runes: orig=1, rev=1, doubleRev=1
        reverse_test.go:34: Before: "\x80", after: "�"
    
FAIL
exit status 1
FAIL	example/fuzz	0.318s


exit status 1


In [21]:
!(cd fuzz;go test -run=FuzzReverse/98fce631eb9c5dd541458bf71a7241469a103001c64053f3a71625b41ccaea5f)

input: "\x80"
input: "�"
--- FAIL: FuzzReverse (0.00s)
    --- FAIL: FuzzReverse/98fce631eb9c5dd541458bf71a7241469a103001c64053f3a71625b41ccaea5f (0.00s)
        reverse_test.go:32: Number of runes: orig=1, rev=1, doubleRev=1
        reverse_test.go:34: Before: "\x80", after: "�"
FAIL
exit status 1
FAIL	example/fuzz	0.132s


exit status 1


In [22]:
!(cd fuzz; ./reset3; go test -fuzz="Fuzz" -fuzztime 30s)

fuzz: elapsed: 0s, gathering baseline coverage: 0/49 completed
fuzz: elapsed: 0s, gathering baseline coverage: 49/49 completed, now fuzzing with 10 workers
fuzz: elapsed: 3s, execs: 1336539 (445432/sec), new interesting: 0 (total: 49)
fuzz: elapsed: 6s, execs: 2784962 (482810/sec), new interesting: 0 (total: 49)
fuzz: elapsed: 9s, execs: 4188005 (467473/sec), new interesting: 0 (total: 49)
fuzz: elapsed: 12s, execs: 5623787 (478738/sec), new interesting: 0 (total: 49)
fuzz: elapsed: 15s, execs: 7079476 (485378/sec), new interesting: 0 (total: 49)
fuzz: elapsed: 18s, execs: 8534969 (485162/sec), new interesting: 0 (total: 49)
fuzz: elapsed: 21s, execs: 10012493 (492352/sec), new interesting: 0 (total: 49)
fuzz: elapsed: 24s, execs: 11448613 (478814/sec), new interesting: 0 (total: 49)
fuzz: elapsed: 27s, execs: 12917083 (489489/sec), new interesting: 0 (total: 49)
fuzz: elapsed: 30s, execs: 14344325 (475794/sec), new interesting: 0 (total: 49)
fuzz: elapsed: 30s, execs: 14344325 (0/sec)

In [ ]:
//package main

import (
    "errors"
    "fmt"
    "unicode/utf8"
)

func main() {
    input := "The quick brown fox jumped over the lazy dog"
    rev, revErr := Reverse(input)
    doubleRev, doubleRevErr := Reverse(rev)
    fmt.Printf("original: %q\n", input)
    fmt.Printf("reversed: %q, err: %v\n", rev, revErr)
    fmt.Printf("reversed again: %q, err: %v\n", doubleRev, doubleRevErr)
}

func Reverse(s string) (string, error) {
    if !utf8.ValidString(s) {
        return s, errors.New("input is not valid UTF-8")
    }
    r := []rune(s)
    for i, j := 0, len(r)-1; i < len(r)/2; i, j = i+1, j-1 {
        r[i], r[j] = r[j], r[i]
    }
    return string(r), nil
}

In [ ]:
package main

import (
    "testing"
    "unicode/utf8"
)

func FuzzReverse(f *testing.F) {
    testcases := []string{"Hello, world", " ", "!12345"}
    for _, tc := range testcases {
        f.Add(tc) // Use f.Add to provide a seed corpus
    }
    f.Fuzz(func(t *testing.T, orig string) {
        rev, err1 := Reverse(orig)
        if err1 != nil {
            return
        }
        doubleRev, err2 := Reverse(rev)
        if err2 != nil {
            return
        }
        if orig != doubleRev {
            t.Errorf("Before: %q, after: %q", orig, doubleRev)
        }
        if utf8.ValidString(orig) && !utf8.ValidString(rev) {
            t.Errorf("Reverse produced invalid UTF-8 string %q", rev)
        }
    })
}